# 3D Ken Burns
This is a reference implementation of 3D Ken Burns Effect from a Single Image [1] using PyTorch. Given a single input image, it animates this still image with a virtual camera scan and zoom subject to motion parallax. Should you be making use of our work, please cite our paper [1].

<a href="https://arxiv.org/abs/1909.05483" rel="Paper"><img src="http://content.sniklaus.com/kenburns/paper.jpg" alt="Paper" width="100%"></a>


> Colab created by [Manuel Romero](https://twitter.com/mrm8488); support him on [Ko-Fi](https://ko-fi.com/mrm8488).
> 
> See advanced settings reference here: https://colab.research.google.com/drive/1W0CkTMaKsmYTEZL-ca7BQK86fybAWA2d?usp=sharing

Quality of Life improvements by [Downy](http://www.twitter.com/tooltrackers).

In [1]:
#@title Check GPU
#@markdown - Tier List: (K80 < T4 < P100 < V100 < A100)
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-1cae71c2-0739-35d2-d6c2-389b7199fe3a)


### 1. Clone the project

In [2]:
!git clone https://github.com/sniklaus/3d-ken-burns.git

Cloning into '3d-ken-burns'...
remote: Enumerating objects: 202, done.
remote: Total 202 (delta 0), reused 0 (delta 0), pack-reused 202
Receiving objects: 100% (202/202), 345.75 KiB | 11.92 MiB/s, done.
Resolving deltas: 100% (126/126), done.


### 2. Setup

In [3]:
#@title Download pretrained models

!wget -O ./3d-ken-burns/models/disparity-estimation.pytorch http://content.sniklaus.com/kenburns/network-disparity.pytorch
!wget -O ./3d-ken-burns/models/disparity-refinement.pytorch http://content.sniklaus.com/kenburns/network-refinement.pytorch
!wget -O ./3d-ken-burns/models/pointcloud-inpainting.pytorch http://content.sniklaus.com/kenburns/network-inpainting.pytorch

--2022-09-21 21:01:32--  http://content.sniklaus.com/kenburns/network-disparity.pytorch
Resolving content.sniklaus.com (content.sniklaus.com)... 88.198.95.97
Connecting to content.sniklaus.com (content.sniklaus.com)|88.198.95.97|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 357438463 (341M) [application/octet-stream]
Saving to: ‘./3d-ken-burns/models/disparity-estimation.pytorch’

./3d-ken-burns/mode 100%[===================>] 340.88M   223MB/s    in 1.5s    

2022-09-21 21:01:34 (223 MB/s) - ‘./3d-ken-burns/models/disparity-estimation.pytorch’ saved [357438463/357438463]

--2022-09-21 21:01:34--  http://content.sniklaus.com/kenburns/network-refinement.pytorch
Resolving content.sniklaus.com (content.sniklaus.com)... 88.198.95.97
Connecting to content.sniklaus.com (content.sniklaus.com)|88.198.95.97|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2438928 (2.3M) [application/octet-stream]
Saving to: ‘./3d-ken-burns/models/disparity-refi

In [4]:
#@title Install Libraries

!pip install chainer
!pip install moviepy
!pip install gevent

import chainer
chainer.print_runtime_info()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 14.8 MB/s 
  Created wheel for chainer: filename=chainer-7.8.1-py3-none-any.whl size=967740 sha256=97335c9c3c8658f4f04405fcc37421aa3de7507b93e7310245ed21897663c242
  Stored in directory: /root/.cache/pip/wheels/c8/6a/6f/fd563166cc597e5206e375ea074ea836e5db5dd58421215672
Successfully built chainer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 15.7 MB/s 
     |████████████████████████████████| 251 kB 70.6 MB/s 


/usr/local/lib/python3.7/dist-packages/chainer/_environment_check.py:75: UserWarning: 
--------------------------------------------------------------------------------
CuPy (cupy-cuda111) version 9.4.0 may not be compatible with this version of Chainer.
Please consider installing the supported version by running:
  $ pip install 'cupy-cuda111>=7.7.0,<8.0.0'

See the following page for more details:
  https://docs.cupy.dev/en/latest/install.html
--------------------------------------------------------------------------------

  requirement=requirement, help=help))


Platform: Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
Chainer: 7.8.1
ChainerX: Not Available
NumPy: 1.21.6
CuPy:
  OS                           : Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
  Python Version               : 3.7.14
  CuPy Version                 : 9.4.0
  CuPy Platform                : NVIDIA CUDA
  NumPy Version                : 1.21.6
  SciPy Version                : 1.7.3
  Cython Build Version         : 0.29.22
  Cython Runtime Version       : 0.29.32
  CUDA Root                    : /usr/local/cuda
  nvcc PATH                    : /usr/local/cuda/bin/nvcc
  CUDA Build Version           : 11010
  CUDA Driver Version          : 11020
  CUDA Runtime Version         : 11010
  cuBLAS Version               : 11300
  cuFFT Version                : 10300
  cuRAND Version               : 10202
  cuSOLVER Version             : (11, 0, 1)
  cuSPARSE Version             : 11300
  NVRTC Version                : (11, 1)
  Thrust Version               : 100910
  CUB Build 

In [5]:
#@title Manage directories

import os
os.environ['CUDA_HOME'] = "/usr/local/cuda"

%cd /content/3d-ken-burns

!rm images/README.md
!rm images/doublestrike.jpg

!mkdir results

/content/3d-ken-burns


### 3. Upload pictures from your filesystem

In [6]:
from google.colab import files

uploaded = files.upload()
for filename in uploaded.keys():
  !mv ./$filename ./images/$filename

Saving batman_solo.jpg to batman_solo.jpg
Saving cleanplate.jpg to cleanplate.jpg
Saving matte.jpg to matte.jpg


In [10]:
#@title Download Autozoom-MEGA from Downy

#        'fltShift': 0.1,
#        'fltZoom': 1000000,
#        'fltSteps': numpy.linspace(-0.5, 5.0, 3000).tolist(),
#        'boolInpaint': False

!wget https://raw.githubusercontent.com/downysoftware/3D-Ken-Burns-MEGA/master/autozoom-MEGA.py

--2022-09-21 21:06:15--  https://raw.githubusercontent.com/downysoftware/3D-Ken-Burns-MEGA/master/autozoom-MEGA.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2797 (2.7K) [text/plain]
Saving to: ‘autozoom-MEGA.py’

autozoom-MEGA.py    100%[===================>]   2.73K  --.-KB/s    in 0s      

2022-09-21 21:06:15 (51.8 MB/s) - ‘autozoom-MEGA.py’ saved [2797/2797]



In [17]:
#@title Fix September update bug

#!pip list --uptodate

!pip install imageio==2.4.1

#!pip install moviepy 0.2.3.5

#!pip uninstall ffmpeg
#!pip install imageio-ffmpeg
#!pip install imageio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 16.4 MB/s 
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303885 sha256=8c5c016c9f7390730fd3e4c8d58ca0313a2394c5fe3e7b1016ce982394a3764a
  Stored in directory: /root/.cache/pip/wheels/46/20/07/7bb9c8c44e6ec2efa60fd0e6280094f53f65f41767ef69a5ee
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0


### 4. Apply (bulk) 3D Ken Burns Effect

In [ ]:
!for image in ./images/*; do python autozoom-MEGA.py --in $image --out ./results/$(basename $image | cut -f1 -d '.').mp4; done

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2990080/45929032 bytes (6.5%)5529600/45929032 bytes (12.0%)7307264/45929032 bytes (15.9%)10108928/45929032 bytes (22.0%)12083200/45929032 bytes (26.3%)14696448/45929032 bytes (32.0%)16138240/45929032 bytes (35.1%)18227200/45929032 bytes (39.7%)20316160/45929032 bytes (44.2%)22118400/45929032 bytes (48.2%)25305088/45929032 bytes (55.1%)27017216/45929032 bytes (58.8%)

#### Now you have all your videos under the folder **results** and you can *manually* right click and download them.

### If you want to *automatically* download all the videos run the following cell:

In [9]:
#@title Download results

for filename in os.listdir("./results"):
  extension = os.path.splitext(filename)[-1].lower()
  if extension == ".mp4":
    files.download("./results/" + filename)

NameError: ignored